## Campbell e Schiller (1991)

Usando dados das LTNs

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

In [2]:
# mostra o dataframe inteiro
# pd.set_option('display.max_rows', None)

Para o caso brasileiro, a base de dados que será usada são os preços da Letra do Tesouro Nacional (LTN), cujo pagamento de R\$ 1000 é feito após $t$ períodos. As LTNs farão o papel dos títulos americanos.

Os dados das LTNs estão em periodicidade mensal (último dia do mês), e abrangem todo o período a partir de Junho de 2012 até Outubro de 2022. Essa base de dados também foi obtida via Reuters com acesso concedido pelo departamento de Economia da PUC-Rio.

In [3]:
# lendo base de dados dos retornos das LTNs
ltn_retorno = pd.read_csv('LTN_retornos.csv', sep=',', index_col=0)

In [4]:
# lendo o dicionário com o significado de cada variável do dataframe ltn_retorno
ltn_dict = pd.read_csv('LTN_dict.csv', sep=',', index_col=0)
ltn_dict

,RIC,Curve Type,Country / Region,Currency,Tenor,Constituents,Rate Type,Curve / Constituents,Contributor
Name,,,,,,,,,
Brazilian Real Zero Coupon Tomorrow Next Point,BRLPRETNZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,Tomorrow Next,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 1 Week Point,BRLPRE1WZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,1 Week,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 1 Month Point,BRLPRE1MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,1 Month,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 2 Month Point,BRLPRE2MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,2 Month,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 3 Month Point,BRLPRE3MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,3 Month,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 6 Month Point,BRLPRE6MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,6 Month,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 9 Month Point,BRLPRE9MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,9 Month,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 1 Year Point,BRLPRE1YZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,1 Year,NaN,Spot,Constituent,REFINITIV
Brazilian Real Zero Coupon 1 Year 3 Month Point,BRLPRE1Y3MZ=R,Swap Zero Curve Constituent,Brazil,Brazilian Real,15 Month,NaN,Spot,Constituent,REFINITIV


In [5]:
# função que transforma a coluna de datas em datas de fato
ltn_retorno.index = pd.to_datetime(ltn_retorno.index)

In [6]:
ltn_retorno

,BRLPRETNZ=R,BRLPRE1WZ=R,BRLPRE1MZ=R,BRLPRE2MZ=R,BRLPRE3MZ=R,BRLPRE6MZ=R,BRLPRE9MZ=R,BRLPRE1YZ=R,BRLPRE1Y3MZ=R,BRLPRE1Y6MZ=R,...,BRLPRE2Y9MZ=R,BRLPRE3YZ=R,BRLPRE3Y3MZ=R,BRLPRE3Y6MZ=R,BRLPRE3Y9MZ=R,BRLPRE4YZ=R,BRLPRE4Y3MZ=R,BRLPRE4Y6MZ=R,BRLPRE4Y9MZ=R,BRLPRE5YZ=R
2022-10-31,13.6503,13.6520,13.6600,13.6680,13.6830,13.6700,13.4770,13.1330,12.7362,12.3708,...,11.4985,11.4820,11.4756,11.4743,11.4767,11.4810,11.4863,11.4934,11.5040,11.5200
2022-09-30,13.6500,13.6524,13.6680,13.6820,13.6890,13.6880,13.5580,13.2450,12.9344,12.5840,...,11.6156,11.6120,11.6210,11.6353,11.6523,11.6690,11.6832,11.6963,11.7113,11.7310
2022-08-31,13.6507,13.6551,13.6730,13.7080,13.7430,13.7600,13.7380,13.5260,13.2296,12.9223,...,11.9620,11.9130,11.8925,11.8902,11.8956,11.8980,11.8897,11.8767,11.8664,11.8690
2022-07-31,13.1500,13.2074,13.5130,13.6190,13.7240,13.8790,13.9150,13.8230,13.6423,13.4427,...,12.7569,12.7140,12.6944,12.6934,12.7082,12.7370,12.7742,12.8140,12.8460,12.8630
2022-06-30,13.1450,13.1403,13.1440,13.3300,13.4790,13.7820,13.9230,13.9230,13.7418,13.5622,...,12.8624,12.7810,12.7203,12.6843,12.6764,12.6990,12.7526,12.8201,12.8793,12.9120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-04-30,9.1205,9.2502,9.3323,8.8829,8.8447,8.5166,8.3520,8.3265,8.4350,8.5899,...,9.4286,9.5563,9.6779,9.7836,9.8726,9.9437,10.0015,10.0507,10.0985,10.1433
2012-03-31,9.9405,9.7844,9.4422,9.4671,9.2289,9.1058,9.0122,8.9508,9.0529,9.2298,...,10.1583,10.2795,10.3822,10.4712,10.5436,10.6076,10.6677,10.7281,10.7935,10.8589
2012-02-29,10.8254,10.7082,10.2655,9.9632,10.1212,9.6860,9.3760,9.2789,9.3640,9.5152,...,10.2187,10.3283,10.4541,10.5755,10.6830,10.7707,10.8357,10.8856,10.9304,10.9725
2012-01-31,10.8586,10.6552,10.2868,10.2309,10.0195,9.8701,9.6671,9.5462,9.5637,9.6989,...,10.4420,10.5596,10.6588,10.7555,10.8365,10.8963,10.9347,10.9611,10.9846,11.0131


In [7]:
# dropando colunas que não serão usadads
ltn_retorno = ltn_retorno.drop(['BRLPRETNZ=R', 'BRLPRE1WZ=R', 'BRLPRE2Y3MZ=R', 'BRLPRE2Y6MZ=R', 'BRLPRE2Y9MZ=R','BRLPRE3YZ=R', 'BRLPRE3Y3MZ=R', 'BRLPRE3Y6MZ=R', 'BRLPRE3Y9MZ=R', 'BRLPRE4YZ=R', 'BRLPRE4Y3MZ=R', 'BRLPRE4Y6MZ=R', 'BRLPRE4Y9MZ=R', 'BRLPRE5YZ=R'], axis = 1)

In [8]:
# normalizando os retornos das LTNs para decimal
ltn_retorno = ltn_retorno/100

In [9]:
# base de dados pronta!
ltn_retorno

,BRLPRE1MZ=R,BRLPRE2MZ=R,BRLPRE3MZ=R,BRLPRE6MZ=R,BRLPRE9MZ=R,BRLPRE1YZ=R,BRLPRE1Y3MZ=R,BRLPRE1Y6MZ=R,BRLPRE1Y9MZ=R,BRLPRE2YZ=R
2022-10-31,0.136600,0.136680,0.136830,0.136700,0.134770,0.131330,0.127362,0.123708,0.120534,0.117980
2022-09-30,0.136680,0.136820,0.136890,0.136880,0.135580,0.132450,0.129344,0.125840,0.122334,0.119380
2022-08-31,0.136730,0.137080,0.137430,0.137600,0.137380,0.135260,0.132296,0.129223,0.126267,0.123750
2022-07-31,0.135130,0.136190,0.137240,0.138790,0.139150,0.138230,0.136423,0.134427,0.132443,0.130640
2022-06-30,0.131440,0.133300,0.134790,0.137820,0.139230,0.139230,0.137418,0.135622,0.133919,0.132310
...,...,...,...,...,...,...,...,...,...,...
2012-04-30,0.093323,0.088829,0.088447,0.085166,0.083520,0.083265,0.084350,0.085899,0.087645,0.089309
2012-03-31,0.094422,0.094671,0.092289,0.091058,0.090122,0.089508,0.090529,0.092298,0.094342,0.096231
2012-02-29,0.102655,0.099632,0.101212,0.096860,0.093760,0.092789,0.093640,0.095152,0.096842,0.098339
2012-01-31,0.102868,0.102309,0.100195,0.098701,0.096671,0.095462,0.095637,0.096989,0.098707,0.100375


In [10]:
# renomeando as colunas de acordo
ltn_retorno = ltn_retorno.rename(columns={'BRLPRE1MZ=R':'y_t(1)',
                                          'BRLPRE2MZ=R':'y_t(2)',
                                          'BRLPRE3MZ=R':'y_t(3)',
                                          'BRLPRE6MZ=R':'y_t(6)',
                                          'BRLPRE9MZ=R':'y_t(9)',
                                          'BRLPRE1YZ=R':'y_t(12)',
                                          'BRLPRE1Y3MZ=R':'y_t(15)',
                                          'BRLPRE1Y6MZ=R':'y_t(18)',
                                          'BRLPRE1Y9MZ=R':'y_t(21)',
                                          'BRLPRE2YZ=R':'y_t(24)'})

Como $P_t^{(n)} = \ln[B_t^{(n)}]$ e $y_t^{(n)} = \ln[1+r_t^{(n)}]$, somos capazes de recuperar os preços das LTNs através de 

$$
y_t^{(n)} = -\frac{P_t^{(n)}}{n}
$$

$$
P_t^{(n)} = -n\cdot y_t^{(n)}
$$

Embora não haja essa necessidade uma vez que as regressões são feitas utilizandos as yields.

Precisamos definir a yield $y_{t+1}^{(n-1)}$ como o rendimento de comprar uma LTN em $t+1$ que paga R\$1 em $t+1+(n-1) = t+n$

Vale notar que aqui, para alguns casos, temos uma periodicidade diferente em relação ao futuro DI, por isso precisamos fazer esse ajuste

In [11]:
# defasagens com periodicidade mensal
ltn_retorno['y_t+1(2-1)'] = ltn_retorno['y_t(1)'].shift(1)
ltn_retorno['y_t+1(3-1)'] = ltn_retorno['y_t(2)'].shift(1)

In [12]:
# defasagens com periodicidade trimestral
ltn_retorno['y_t+3(6-3)'] = ltn_retorno['y_t(3)'].shift(3)
ltn_retorno['y_t+3(9-3)'] = ltn_retorno['y_t(6)'].shift(3)
ltn_retorno['y_t+3(12-3)'] = ltn_retorno['y_t(9)'].shift(3)
ltn_retorno['y_t+3(15-3)'] = ltn_retorno['y_t(12)'].shift(3)
ltn_retorno['y_t+3(18-3)'] = ltn_retorno['y_t(15)'].shift(3)
ltn_retorno['y_t+3(21-3)'] = ltn_retorno['y_t(18)'].shift(3)
ltn_retorno['y_t+3(24-3)'] = ltn_retorno['y_t(21)'].shift(3)

In [13]:
ltn_retorno

,y_t(1),y_t(2),y_t(3),y_t(6),y_t(9),y_t(12),y_t(15),y_t(18),y_t(21),y_t(24),y_t+1(2-1),y_t+1(3-1),y_t+3(6-3),y_t+3(9-3),y_t+3(12-3),y_t+3(15-3),y_t+3(18-3),y_t+3(21-3),y_t+3(24-3)
2022-10-31,0.136600,0.136680,0.136830,0.136700,0.134770,0.131330,0.127362,0.123708,0.120534,0.117980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-30,0.136680,0.136820,0.136890,0.136880,0.135580,0.132450,0.129344,0.125840,0.122334,0.119380,0.136600,0.136680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-31,0.136730,0.137080,0.137430,0.137600,0.137380,0.135260,0.132296,0.129223,0.126267,0.123750,0.136680,0.136820,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-07-31,0.135130,0.136190,0.137240,0.138790,0.139150,0.138230,0.136423,0.134427,0.132443,0.130640,0.136730,0.137080,0.136830,0.136700,0.134770,0.131330,0.127362,0.123708,0.120534
2022-06-30,0.131440,0.133300,0.134790,0.137820,0.139230,0.139230,0.137418,0.135622,0.133919,0.132310,0.135130,0.136190,0.136890,0.136880,0.135580,0.132450,0.129344,0.125840,0.122334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-04-30,0.093323,0.088829,0.088447,0.085166,0.083520,0.083265,0.084350,0.085899,0.087645,0.089309,0.088239,0.088801,0.077763,0.074246,0.073377,0.074538,0.076579,0.078730,0.080587
2012-03-31,0.094422,0.094671,0.092289,0.091058,0.090122,0.089508,0.090529,0.092298,0.094342,0.096231,0.093323,0.088829,0.082769,0.077425,0.075624,0.075823,0.076874,0.078317,0.079833
2012-02-29,0.102655,0.099632,0.101212,0.096860,0.093760,0.092789,0.093640,0.095152,0.096842,0.098339,0.094422,0.094671,0.085793,0.081212,0.079692,0.079635,0.080489,0.081787,0.083217
2012-01-31,0.102868,0.102309,0.100195,0.098701,0.096671,0.095462,0.095637,0.096989,0.098707,0.100375,0.102655,0.099632,0.088447,0.085166,0.083520,0.083265,0.084350,0.085899,0.087645


Defina $s_t^{(n)} = y_t^{(n)} - y_t^{(1)}$

In [14]:
ltn_retorno['s_t(2)'] = ltn_retorno['y_t(2)'] - ltn_retorno['y_t(1)']
ltn_retorno['s_t(3)'] = ltn_retorno['y_t(3)'] - ltn_retorno['y_t(1)']
ltn_retorno['s_t(6)'] = ltn_retorno['y_t(6)'] - ltn_retorno['y_t(1)']
ltn_retorno['s_t(9)'] = ltn_retorno['y_t(9)'] - ltn_retorno['y_t(1)']
ltn_retorno['s_t(12)'] = ltn_retorno['y_t(12)'] - ltn_retorno['y_t(1)']
ltn_retorno['s_t(15)'] = ltn_retorno['y_t(15)'] - ltn_retorno['y_t(1)']
ltn_retorno['s_t(18)'] = ltn_retorno['y_t(18)'] - ltn_retorno['y_t(1)']
ltn_retorno['s_t(21)'] = ltn_retorno['y_t(21)'] - ltn_retorno['y_t(1)']
ltn_retorno['s_t(24)'] = ltn_retorno['y_t(24)'] - ltn_retorno['y_t(1)']

O primeiro modelo de Campbell e Schiller (1991) é mostrado a seguir

$$
y_{t+1}^{(n-1)} - y_t^{(n)} = \alpha_n + \beta_n \frac{s_t^{(n)}}{n-1} + \epsilon_t^{(n)}
$$

In [15]:
ltn_retorno['y'] = ltn_retorno['y_t+1(2-1)'] - ltn_retorno['y_t(2)']
ltn_retorno['x'] = ltn_retorno['s_t(2)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n = 2$, a regressão é

$$
y_{t+1}^{(1)} - y_t^{(2)} = \alpha_2 + \beta_2 s_t^{(2)} + \epsilon_t^{(2)}
$$

In [16]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.527
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     142.8
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.42e-22
Time:                        19:43:13   Log-Likelihood:                 647.27
No. Observations:                 130   AIC:                            -1291.
Df Residuals:                     128   BIC:                            -1285.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0001      0.000     -0.752      0.4

In [17]:
ltn_retorno['y'] = ltn_retorno['y_t+1(3-1)'] - ltn_retorno['y_t(3)']
ltn_retorno['x'] = ltn_retorno['s_t(3)']/2
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n = 3$, a regressão é

$$
y_{t+1}^{(2)} - y_t^{(3)} = \alpha_3 + \beta_3 \frac{s_t^{(3)}}{2} + \epsilon_t^{(3)}
$$

In [18]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.426
Method:                 Least Squares   F-statistic:                     96.68
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           2.44e-17
Time:                        19:43:14   Log-Likelihood:                 618.80
No. Observations:                 130   AIC:                            -1234.
Df Residuals:                     128   BIC:                            -1228.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0005      0.000     -2.568      0.0

Como as defasagens agora são de três meses, temos que o n dividindo o regressor deve ser ajustado para trimestral. O restante da notação será mantido para que não haja confusão

In [19]:
ltn_retorno['y'] = ltn_retorno['y_t+3(6-3)'] - ltn_retorno['y_t(6)']
ltn_retorno['x'] = ltn_retorno['s_t(6)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n = 6$, a regressão é

$$
y_{t+3}^{(3)} - y_t^{(6)} = \alpha_6 + \beta_6 s_t^{(6)} + \epsilon_t^{(6)}
$$

In [20]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.511
Method:                 Least Squares   F-statistic:                     133.8
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.57e-21
Time:                        19:43:14   Log-Likelihood:                 483.84
No. Observations:                 128   AIC:                            -963.7
Df Residuals:                     126   BIC:                            -958.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0014      0.001     -2.842      0.0

In [21]:
ltn_retorno['y'] = ltn_retorno['y_t+3(9-3)'] - ltn_retorno['y_t(9)']
ltn_retorno['x'] = ltn_retorno['s_t(9)']/2
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n = 9$, a regressão é

$$
y_{t+3}^{(6)} - y_t^{(9)} = \alpha_9 + \beta_9 \frac{s_t^{(9)}}{2} + \epsilon_t^{(9)}
$$

In [22]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.395
Model:                            OLS   Adj. R-squared:                  0.390
Method:                 Least Squares   F-statistic:                     82.29
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.97e-15
Time:                        19:43:14   Log-Likelihood:                 444.76
No. Observations:                 128   AIC:                            -885.5
Df Residuals:                     126   BIC:                            -879.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0016      0.001     -2.279      0.0

In [23]:
ltn_retorno['y'] = ltn_retorno['y_t+3(12-3)'] - ltn_retorno['y_t(12)']
ltn_retorno['x'] = ltn_retorno['s_t(12)']/3
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n = 12$, a regressão é

$$
y_{t+3}^{(9)} - y_t^{(12)} = \alpha_{12} + \beta_{12} \frac{s_t^{(12)}}{3} + \epsilon_t^{(12)}
$$

In [24]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.327
Model:                            OLS   Adj. R-squared:                  0.322
Method:                 Least Squares   F-statistic:                     61.18
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.81e-12
Time:                        19:43:15   Log-Likelihood:                 423.19
No. Observations:                 128   AIC:                            -842.4
Df Residuals:                     126   BIC:                            -836.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0020      0.001     -2.374      0.0

In [25]:
ltn_retorno['y'] = ltn_retorno['y_t+3(15-3)'] - ltn_retorno['y_t(15)']
ltn_retorno['x'] = ltn_retorno['s_t(15)']/4
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n = 15$, a regressão é

$$
y_{t+3}^{(12)} - y_t^{(15)} = \alpha_{15} + \beta_{15} \frac{s_t^{(15)}}{4} + \epsilon_t^{(15)}
$$

In [26]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.279
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     48.70
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.51e-10
Time:                        19:43:15   Log-Likelihood:                 411.79
No. Observations:                 128   AIC:                            -819.6
Df Residuals:                     126   BIC:                            -813.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0024      0.001     -2.651      0.0

In [27]:
ltn_retorno['y'] = ltn_retorno['y_t+3(18-3)'] - ltn_retorno['y_t(18)']
ltn_retorno['x'] = ltn_retorno['s_t(18)']/5
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n = 18$, a regressão é

$$
y_{t+3}^{(15)} - y_t^{(18)} = \alpha_{18} + \beta_{18} \frac{s_t^{(18)}}{5} + \epsilon_t^{(18)}
$$

In [28]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     35.21
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           2.68e-08
Time:                        19:43:15   Log-Likelihood:                 405.22
No. Observations:                 128   AIC:                            -806.4
Df Residuals:                     126   BIC:                            -800.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0028      0.001     -2.863      0.0

In [29]:
ltn_retorno['y'] = ltn_retorno['y_t+3(21-3)'] - ltn_retorno['y_t(21)']
ltn_retorno['x'] = ltn_retorno['s_t(21)']/6
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n = 21$, a regressão é

$$
y_{t+3}^{(18)} - y_t^{(21)} = \alpha_{21} + \beta_{21} \frac{s_t^{(21)}}{6} + \epsilon_t^{(21)}
$$

In [30]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     22.94
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           4.62e-06
Time:                        19:43:16   Log-Likelihood:                 398.12
No. Observations:                 128   AIC:                            -792.2
Df Residuals:                     126   BIC:                            -786.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0029      0.001     -2.704      0.0

In [31]:
ltn_retorno['y'] = ltn_retorno['y_t+3(24-3)'] - ltn_retorno['y_t(24)']
ltn_retorno['x'] = ltn_retorno['s_t(24)']/7
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n = 24$, a regressão é

$$
y_{t+3}^{(21)} - y_t^{(24)} = \alpha_{24} + \beta_{24} \frac{s_t^{(24)}}{7} + \epsilon_t^{(24)}
$$

In [32]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     14.50
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           0.000217
Time:                        19:43:16   Log-Likelihood:                 392.40
No. Observations:                 128   AIC:                            -780.8
Df Residuals:                     126   BIC:                            -775.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0028      0.001     -2.438      0.0

O segundo modelo é

$$
\sum_{i=1}^{n} \left(1 - \frac{i}{n}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(n)} - y_t^{(1)}] + v_t^{(n)}
$$

Precisamos então de mais algumas yields. Defina $y_{t+i}^{(1)}$ como o retorno da LTN comprada em $t+i$ que paga R\$1 em $t+i+1$

In [33]:
ltn_retorno['y_t+2(1)'] = ltn_retorno['y_t(1)'].shift(2)
ltn_retorno['y_t+3(1)'] = ltn_retorno['y_t(1)'].shift(3)
ltn_retorno['y_t+4(1)'] = ltn_retorno['y_t(1)'].shift(4)
ltn_retorno['y_t+5(1)'] = ltn_retorno['y_t(1)'].shift(5)
ltn_retorno['y_t+6(1)'] = ltn_retorno['y_t(1)'].shift(6)
ltn_retorno['y_t+7(1)'] = ltn_retorno['y_t(1)'].shift(7)
ltn_retorno['y_t+8(1)'] = ltn_retorno['y_t(1)'].shift(8)
ltn_retorno['y_t+9(1)'] = ltn_retorno['y_t(1)'].shift(9)
ltn_retorno['y_t+10(1)'] = ltn_retorno['y_t(1)'].shift(10)
ltn_retorno['y_t+11(1)'] = ltn_retorno['y_t(1)'].shift(11)

In [34]:
ltn_retorno['y'] = (1/2)*(ltn_retorno['y_t+1(2-1)'] - ltn_retorno['y_t(1)'])
ltn_retorno['x'] = ltn_retorno['s_t(2)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n=2$, a regressão é

$$
\sum_{i=1}^{2} \left(1 - \frac{i}{2}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(2)} - y_t^{(1)}] + v_t^{(2)}
$$

$$
\frac{1}{2}\left(y_{t+1}^{(1)} - y_{t}^{(1)}\right) = \alpha + \beta[y_t^{(2)} - y_t^{(1)}] + v_t^{(2)}
$$

In [35]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     693.2
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.66e-53
Time:                        19:43:16   Log-Likelihood:                 737.38
No. Observations:                 130   AIC:                            -1471.
Df Residuals:                     128   BIC:                            -1465.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -5.544e-05   7.37e-05     -0.752      0.4

In [36]:
ltn_retorno['y'] = (2/3)*(ltn_retorno['y_t+1(2-1)'] - ltn_retorno['y_t(1)']) + (1/3)*(ltn_retorno['y_t+2(1)']-ltn_retorno['y_t+1(2-1)'])
ltn_retorno['x'] = ltn_retorno['s_t(3)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n=3$, a regressão é

$$
\sum_{i=1}^{3} \left(1 - \frac{i}{3}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(3)} - y_t^{(1)}] + v_t^{(3)}
$$

$$
\frac{2}{3}\left(y_{t+1}^{(1)} - y_{t}^{(1)}\right) + \frac{1}{3}\left(y_{t+2}^{(1)} - y_{t+1}^{(1)}\right) = \alpha + \beta[y_t^{(3)} - y_t^{(1)}] + v_t^{(3)}
$$

In [37]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     794.0
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.74e-56
Time:                        19:43:17   Log-Likelihood:                 660.93
No. Observations:                 129   AIC:                            -1318.
Df Residuals:                     127   BIC:                            -1312.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0003      0.000     -2.697      0.0

In [38]:
ltn_retorno['y'] = (5/6)*(ltn_retorno['y_t+1(2-1)'] - ltn_retorno['y_t(1)']) + (4/6)*(ltn_retorno['y_t+2(1)']-ltn_retorno['y_t+1(2-1)']) + (3/6)*(ltn_retorno['y_t+3(1)']-ltn_retorno['y_t+2(1)']) + (2/6)*(ltn_retorno['y_t+4(1)']-ltn_retorno['y_t+3(1)']) + (1/6)*(ltn_retorno['y_t+5(1)']-ltn_retorno['y_t+4(1)'])
ltn_retorno['x'] = ltn_retorno['s_t(6)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n=6$, a regressão é

$$
\sum_{i=1}^{6} \left(1 - \frac{i}{6}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(6)} - y_t^{(1)}] + v_t^{(6)}
$$

$$
\frac{5}{6}\left(y_{t+1}^{(1)} - y_{t}^{(1)}\right) + \frac{4}{6}\left(y_{t+2}^{(1)} - y_{t+1}^{(1)}\right) + \frac{3}{6}\left(y_{t+3}^{(1)} - y_{t+2}^{(1)}\right) + \frac{2}{6}\left(y_{t+4}^{(1)} - y_{t+3}^{(1)}\right) + \frac{1}{6}\left(y_{t+5}^{(1)} - y_{t+4}^{(1)}\right) = \alpha + \beta[y_t^{(6)} - y_t^{(1)}] + v_t^{(6)}
$$

In [39]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.849
Method:                 Least Squares   F-statistic:                     704.0
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           5.81e-53
Time:                        19:43:17   Log-Likelihood:                 532.05
No. Observations:                 126   AIC:                            -1060.
Df Residuals:                     124   BIC:                            -1054.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0011      0.000     -3.492      0.0

In [40]:
ltn_retorno['y'] = (8/9)*(ltn_retorno['y_t+1(2-1)'] - ltn_retorno['y_t(1)']) + (7/9)*(ltn_retorno['y_t+2(1)']-ltn_retorno['y_t+1(2-1)']) + (6/9)*(ltn_retorno['y_t+3(1)']-ltn_retorno['y_t+2(1)']) + (5/9)*(ltn_retorno['y_t+4(1)']-ltn_retorno['y_t+3(1)']) + (4/9)*(ltn_retorno['y_t+5(1)']-ltn_retorno['y_t+4(1)']) + (3/9)*(ltn_retorno['y_t+6(1)']-ltn_retorno['y_t+5(1)']) + (2/9)*(ltn_retorno['y_t+7(1)']-ltn_retorno['y_t+6(1)']) + (1/9)*(ltn_retorno['y_t+8(1)']-ltn_retorno['y_t+7(1)'])
ltn_retorno['x'] = ltn_retorno['s_t(9)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n=9$, a regressão é

$$
\sum_{i=1}^{9} \left(1 - \frac{i}{9}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(9)} - y_t^{(1)}] + v_t^{(9)}
$$

In [41]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.815
Model:                            OLS   Adj. R-squared:                  0.813
Method:                 Least Squares   F-statistic:                     532.6
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           3.86e-46
Time:                        19:43:17   Log-Likelihood:                 453.18
No. Observations:                 123   AIC:                            -902.4
Df Residuals:                     121   BIC:                            -896.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0021      0.001     -3.631      0.0

In [42]:
ltn_retorno['y'] = (11/12)*(ltn_retorno['y_t+1(2-1)'] - ltn_retorno['y_t(1)']) + (10/12)*(ltn_retorno['y_t+2(1)']-ltn_retorno['y_t+1(2-1)']) + (9/12)*(ltn_retorno['y_t+3(1)']-ltn_retorno['y_t+2(1)']) + (8/12)*(ltn_retorno['y_t+4(1)']-ltn_retorno['y_t+3(1)']) + (7/12)*(ltn_retorno['y_t+5(1)']-ltn_retorno['y_t+4(1)']) + (6/12)*(ltn_retorno['y_t+6(1)']-ltn_retorno['y_t+5(1)']) + (5/12)*(ltn_retorno['y_t+7(1)']-ltn_retorno['y_t+6(1)']) + (4/12)*(ltn_retorno['y_t+8(1)']-ltn_retorno['y_t+7(1)']) + (3/12)*(ltn_retorno['y_t+9(1)']-ltn_retorno['y_t+8(1)']) + (2/12)*(ltn_retorno['y_t+10(1)']-ltn_retorno['y_t+9(1)']) + (1/12)*(ltn_retorno['y_t+11(1)']-ltn_retorno['y_t+10(1)'])
ltn_retorno['x'] = ltn_retorno['s_t(12)']
result = sm.ols(formula="y ~ x", data=ltn_retorno).fit()

Para $n=12$, a regressão é

$$
\sum_{i=1}^{12} \left(1 - \frac{i}{12}\right)\Delta y_{t+i}^{(1)} = \alpha + \beta[y_t^{(12)} - y_t^{(1)}] + v_t^{(12)}
$$

In [43]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.780
Model:                            OLS   Adj. R-squared:                  0.778
Method:                 Least Squares   F-statistic:                     417.8
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           1.40e-40
Time:                        19:43:18   Log-Likelihood:                 399.76
No. Observations:                 120   AIC:                            -795.5
Df Residuals:                     118   BIC:                            -789.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0033      0.001     -4.047      0.0